In [1]:
import jpype
import jpype.imports
from jpype.types import *
import numpy as np

print(jpype.getDefaultJVMPath())

jpype.startJVM(jpype.getDefaultJVMPath(),"-ea", "-Djava.class.path=%s" % ('/Users/apimienta/Documents/Minesweeper AI/train'))

MinesweeperInterface = jpype.JClass("MinesweeperInterface")
MinesweeperGame = jpype.JClass("MinesweeperInterface$MinesweeperGame")

/Library/Java/JavaVirtualMachines/jdk1.8.0_102.jdk/Contents/Home/jre/lib/jli/libjli.dylib


In [7]:
"""Generate game and return a tuple of (game, interface)"""
def generate_new_game():
    new_interface = MinesweeperInterface()
    new_game = MinesweeperGame(new_interface)
    
    return (new_game, new_interface)

"""dels the game for garbage collection"""
def destroy_game(interface, game):
    del interface
    del game
    
"""Get board of game as numpy array"""
def get_np_board(game):
    j_board = game.getPlayerBoard()
    return np.array([list(row) for row in j_board])

"""Click at index [x][y] and return whether or not it was a valid click"""
def click_game(game, x, y):
    return game.click(x, y)
    
new_game, new_interface = generate_new_game()
print(new_game.click(3, 3))

print(get_np_board(new_game))

destroy_game(new_game, new_interface)

False
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [3]:
jpype.shutdownJVM()